In [1]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use agent operations with the 
    Azure AI Search tool from the Azure Agents service using a synchronous client.
    To learn how to set up an Azure AI Search resource,
    visit https://learn.microsoft.com/azure/search/search-get-started-portal

USAGE:
    Before running the sample:
    Set these environment variables with your own values:
    1) AZURE_AISTUDIO_PROJECT_CONN_STRING - The project connection string, as found in the overview page of your
       Azure AI Foundry project.
    2) AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in 
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_AI_SEARCH_CONNECTION_ID - The connection ID of the Azure AI Search resource, as found in the "Connections" tab
"""
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool, AzureAISearchTool, ConnectionType, FunctionTool, ToolSet
from azure.identity import DefaultAzureCredential
from typing import Any, Callable, Set, Dict, List, Optional
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from azure.identity import DefaultAzureCredential

load_dotenv()
AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME = "gpt-4o"
AZURE_AISTUDIO_PROJECT_CONN_STRING = os.getenv("NL_TO_KQL_AZURE_AISTUDIO_PROJECT_CONN_STRING")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=AZURE_AISTUDIO_PROJECT_CONN_STRING,
)

KUSTO_URI = os.getenv("NL_TO_KQL_KUSTO_URI")
KUSTO_DATABASE = os.getenv("NL_TO_KQL_KUSTO_DATABASE")
KUSTO_TABLE = os.getenv("NL_TO_KQL_KUSTO_TABLE")

try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    print(ex)

In [2]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

kcsb = KustoConnectionStringBuilder.with_az_cli_authentication(KUSTO_URI)
print(kcsb)
client = KustoClient(kcsb)
kusto_db = KUSTO_DATABASE

Data Source=https://trd-0gy3f5j482jxjzk4xj.z4.kusto.fabric.microsoft.com;Initial Catalog=NetDefaultDB;AAD Federated Security=True;Authority Id=organizations;AZ CLI=True


In [3]:
def execute_query(kusto_query: str):
    response = client.execute(kusto_db, kusto_query)
    df = dataframe_from_result_table(response.primary_results[0])
    df
    return df


def get_table_creation_kql() -> str:
    table_schema = ".create-merge table Stocks (Date:datetime, Open:real, High:real, Low:real, Close:real, AdjClose:real, Volume:string, Ticker:string) "
    return table_schema

In [4]:
from string import Template

def load_template(filename: str) -> Template:
    file = open(filename, "r")
    content = file.read()
    file.close()
    template = Template(content)
    return template

In [9]:
def get_schema_prompt() -> str:
    tbl_creation_kql = get_table_creation_kql()
    template_file_name = "./prompts/schema_prompt.txt"
    template = load_template(template_file_name)
    prompt = template.substitute(tbl_creation_kql=tbl_creation_kql)
    return prompt

In [10]:
def read_from_db(kql_query:str) -> str:
    """
    run the query on the database and return the result

    :param kql_query (str): The kql to run on the database.
    :return: The query result
    """
    query_result = execute_query(kql_query)
    return query_result.to_json()

user_functions: Set[Callable[..., Any]] = {
    read_from_db
}

# Initialize agent toolset with user functions and code interpreter
functions = FunctionTool(user_functions)
code_interpreter = CodeInterpreterTool()

toolset = ToolSet()
toolset.add(functions)
toolset.add(code_interpreter)

agent = project_client.agents.create_agent(
    model=AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME,
    name="kql-assistant",
    instructions=get_schema_prompt(),
    toolset=toolset,
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_tfniYmnGChpsv6oVmeCrIp9K


In [11]:
# Create thread for communication
def call_agent(question):
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=question
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
        # Fetch and log all messages
        
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # print(messages["data"][0]["content"][0]["text"]["value"])
    # Fetch and log all messages in chronological order
    messages_data = messages["data"]

    # Sort messages by creation time (ascending)
    sorted_messages = sorted(messages_data, key=lambda x: x["created_at"])

    print("\n--- Thread Messages (sorted) ---")
    for msg in sorted_messages:
        role = msg["role"].upper()
        # Each 'content' is a list; get the first text block if present
        content_blocks = msg.get("content", [])
        text_value = ""
        if content_blocks and content_blocks[0]["type"] == "text":
            text_value = content_blocks[0]["text"]["value"]
        print(f"{role}: {text_value}")

In [12]:
call_agent("What is the average price for each stock symbol in the February 2013?")

Created thread, ID: thread_pYHDwG64mE404g2NwnWtntnw
Created message, ID: msg_IlFyIWhi3hsfbBwGXarmPHyg
Run finished with status: completed
Messages: {'object': 'list', 'data': [{'id': 'msg_taxAoMsIFrwsYX4M0uP2hRsx', 'object': 'thread.message', 'created_at': 1739808898, 'assistant_id': 'asst_tfniYmnGChpsv6oVmeCrIp9K', 'thread_id': 'thread_pYHDwG64mE404g2NwnWtntnw', 'run_id': 'run_7qDxRnRPMSKvUz1YVFC4G78r', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "Here is the average closing price for each stock symbol in February 2013:\n\n- AAPL: $16.31\n- ABBV: $36.67\n- ABC: $46.66\n- ABT: $34.29\n- ACGL: $15.99\n- AEE: $33.09\n- AEP: $45.33\n- AFL: $25.11\n- AIG: $38.36\n- AIZ: $40.35\n- AJG: $38.28\n- AKAM: $37.78\n- ALB: $63.73\n- ALL: $45.55\n- AMAT: $13.54\n- AMCR: $9.32\n- AMD: $2.62\n- AMGN: $86.43\n- AMP: $67.34\n- AMT: $75.50\n- AMZN: $13.18\n- ANSS: $75.23\n- AON: $57.82\n- APA: $79.77\n- APD: $80.73\n- APTV: $33.00\n- ARE: $71.96\n- ATVI: $13.50\n- ATO: $37.85\n- 

In [13]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
